In [28]:
# Import libraries
import pandas as pd
import numpy as np
import yfinance as yf
import statsmodels.api as sm
from statsmodels.tsa.api import ARIMA
from arch import arch_model
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge, LogisticRegression
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import accuracy_score, mean_squared_error, roc_auc_score
from sklearn.preprocessing import StandardScaler
from hmmlearn.hmm import GaussianHMM
import shap
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

# Configure settings
warnings.filterwarnings('ignore')
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

In [29]:
def ingest_data():
    """
    Ingests raw time series data. Downloads daily data from Yahoo Finance and
    simulates realistic weekly and monthly macroeconomic data.
    """
    from datetime import datetime

# Here we have the tickers from yfinance, maybe switch soon to a more reliable source.
    tickers = {
        'CL=F': 'wti_price',   # WTI Crude Oil Futures
        'BZ=F': 'brent_price', # Brent Crude Oil Futures
        'DX-Y.NYB': 'dxy',     # US Dollar Index
        '^TNX': '10y_yield',   # 10-Year Treasury Yield
        '^VIX': 'vix'          # CBOE Volatility Index
    }


    end=datetime.now().strftime('%Y-%m-%d')
    # From 2005 to today
    data = yf.download(list(tickers.keys()), start='2007-07-30', end=end, progress=False) # just for the bar
    # We're only working with the price at the end of the day, so this will be measured on the scale of days.
    data = data['Close'] # The price is already adjusted for splits and dividends



    
    data = data.rename(columns=tickers)


    # We can't have any missing values, so we'll forward fill them.
    data = data.ffill().dropna() # Forward fill to handle non-trading days

    # --- Simulate Weekly Data (EIA Inventories) ---
    weekly_dates = pd.date_range(start=data.index.min(), end=data.index.max(), freq='W-WED') # We're using the end of the week as the measurement point.

    # Ill have some random data here, but in reality this will be the EIA Inventories data, this will just trend up.
    eia_inventories = pd.Series(
        500 + (np.random.randn(len(weekly_dates)) * 10).cumsum(),
        index=weekly_dates, name='eia_inventories'
    )

    # --- Simulate Monthly Data (CPI, Industrial Production) ---
    monthly_dates = pd.date_range(start=data.index.min(), end=data.index.max(), freq='M')
    cpi = pd.Series(
        250 + (np.random.randn(len(monthly_dates)) * 0.5).cumsum(),
        index=monthly_dates, name='cpi'
    ) # We use 0.5 here because it represents inflation in a way, which is less volatile than the other data.


    industrial_production = pd.Series(
        100 + (np.random.randn(len(monthly_dates)) * 0.2).cumsum(),
        index=monthly_dates, name='industrial_production'
    ) # Very stable data, so we use 0.2.

    # Merge all datasets into a single daily-frequency DataFrame
    df = data.copy()
    df = df.merge(eia_inventories, how='left', left_index=True, right_index=True)
    df = df.merge(cpi, how='left', left_index=True, right_index=True)
    df = df.merge(industrial_production, how='left', left_index=True, right_index=True)

    # --- Data Catalog ---
    metadata = {
        'wti_price': {'frequency': 'daily', 'source': 'Yahoo Finance', 'publication_lag': '0D'},
        'brent_price': {'frequency': 'daily', 'source': 'Yahoo Finance', 'publication_lag': '0D'},
        'dxy': {'frequency': 'daily', 'source': 'Yahoo Finance', 'publication_lag': '0D'},
        '10y_yield': {'frequency': 'daily', 'source': 'Yahoo Finance', 'publication_lag': '0D'},
        'vix': {'frequency': 'daily', 'source': 'Yahoo Finance', 'publication_lag': '0D'},
        'eia_inventories': {'frequency': 'weekly', 'source': 'Simulated', 'publication_lag': '5B'},
        'cpi': {'frequency': 'monthly', 'source': 'Simulated', 'publication_lag': '10B'},
        'industrial_production': {'frequency': 'monthly', 'source': 'Simulated', 'publication_lag': '12B'},
    }

    print("Data ingestion complete.")
    return df, metadata

# Execute data ingestion
raw_data, metadata = ingest_data()
print("Raw Data Head:")
print(raw_data.head())

Data ingestion complete.
Raw Data Head:
            brent_price  wti_price        dxy  10y_yield        vix  \
Date                                                                  
2007-07-30    75.739998  76.830002  80.849998      4.804  20.870001   
2007-07-31    77.050003  78.209999  80.769997      4.771  23.520000   
2007-08-01    75.349998  76.529999  80.870003      4.759  23.670000   
2007-08-02    75.760002  76.860001  80.709999      4.753  21.219999   
2007-08-03    74.750000  75.480003  80.180000      4.700  25.160000   

            eia_inventories         cpi  industrial_production  
Date                                                            
2007-07-30              NaN         NaN                    NaN  
2007-07-31              NaN  250.750811             100.132497  
2007-08-01       502.074518         NaN                    NaN  
2007-08-02              NaN         NaN                    NaN  
2007-08-03              NaN         NaN                    NaN  


In [30]:
# Free Historical Weather Data Integration
# Install required packages for NOAA weather data
%pip install requests --quiet

Note: you may need to restart the kernel to use updated packages.


In [31]:
# Open-Meteo setup: city coordinates and fetch helper
import requests
from datetime import datetime, timedelta

city_coords = {
    # US oil hubs and major cities
    'Houston': (29.7604, -95.3698),
    'Dallas': (32.7767, -96.7970),
    'Denver': (39.7392, -104.9903),
    'New York': (40.7128, -74.0060),
    'Los Angeles': (34.0522, -118.2437),
    'Chicago': (41.8781, -87.6298),
    'London': (51.5074, -0.1278),
    'Tokyo': (35.6762, 139.6503),
}


def fetch_weather_data(city_name, lat, lon, start_date, end_date, timezone="UTC"):
    url = "https://archive-api.open-meteo.com/v1/archive"
    params = {
        "latitude": lat,
        "longitude": lon,
        "start_date": start_date,
        "end_date": end_date,
        "daily": [
            "temperature_2m_mean",
            "precipitation_sum",
            "wind_speed_10m_max"
        ],
        "timezone": timezone
    }
    resp = requests.get(url, params=params)
    resp.raise_for_status()
    data = resp.json()
    if "daily" not in data:
        return pd.DataFrame()
    df = pd.DataFrame(data["daily"])  # columns: time, temperature_2m_mean, precipitation_sum, wind_speed_10m_max
    df["city"] = city_name
    return df

print("Open-Meteo setup ready")

# Chunked fetch to avoid long single requests
from datetime import date as _date
from dateutil.relativedelta import relativedelta as _relativedelta
import time as _time

def _iter_date_chunks(start_date: str, end_date: str, chunk_days: int = 365):
    start = pd.to_datetime(start_date).date()
    end = pd.to_datetime(end_date).date()
    cur = start
    delta = pd.Timedelta(days=chunk_days)
    while cur <= end:
        nxt = min(pd.to_datetime(cur) + delta, pd.to_datetime(end))
        yield cur.strftime('%Y-%m-%d'), nxt.strftime('%Y-%m-%d')
        # advance by one day beyond nxt to avoid overlap
        cur = (pd.to_datetime(nxt) + pd.Timedelta(days=1)).date()

def fetch_city_weather_batched(city_name, lat, lon, start_date, end_date, timezone="UTC", chunk_days=365, sleep_sec=0.2):
    frames = []
    for s, e in _iter_date_chunks(start_date, end_date, chunk_days):
        df_part = fetch_weather_data(city_name, lat, lon, s, e, timezone=timezone)
        if not df_part.empty:
            frames.append(df_part)
        _time.sleep(sleep_sec)
    if frames:
        return pd.concat(frames, ignore_index=True)
    return pd.DataFrame()


Open-Meteo setup ready


In [32]:
# Open-Meteo: fetch weather for analysis
def get_weather_data_for_analysis(start_date, end_date, cities=None, usa_only=False, chunk_days=365, sleep_sec=0.2):
    """Fetch Open-Meteo daily weather for given cities and date range with chunking."""
    if usa_only or not cities:
        target_cities = ['Houston', 'Dallas', 'Denver', 'New York', 'Los Angeles', 'Chicago']
    else:
        target_cities = [c for c in cities if c in city_coords]
        if not target_cities:
            target_cities = ['Houston', 'Dallas', 'Denver', 'New York', 'Los Angeles', 'Chicago']
    frames = []
    for c in target_cities:
        lat, lon = city_coords[c]
        df_c = fetch_city_weather_batched(c, lat, lon, start_date, end_date, timezone="UTC", chunk_days=chunk_days, sleep_sec=sleep_sec)
        if not df_c.empty:
            frames.append(df_c)
    if frames:
        out = pd.concat(frames, ignore_index=True)
        # rename and standardize
        out = out.rename(columns={
            'time': 'date',
            'temperature_2m_mean': 'temp_mean_c',
            'precipitation_sum': 'precip_mm',
            'wind_speed_10m_max': 'wind_max_ms'
        })
        out['date'] = pd.to_datetime(out['date'])
        return out
    return pd.DataFrame()

# Example usage - get weather data for the last 30 days
end_date = datetime.now().strftime('%Y-%m-%d')
start_date = "2007-07-30"

print("Example: Getting weather data for major US cities (last 30 days)")
print("=" * 60)


Example: Getting weather data for major US cities (last 30 days)


In [33]:
# Demo: Get weather data for specific cities (Open-Meteo)
demo_cities = ['New York', 'Los Angeles', 'Chicago', 'Houston', 'London', 'Tokyo']

weather_data = get_weather_data_for_analysis(start_date, end_date, cities=demo_cities)

if not weather_data.empty:
    print("Weather Data Sample:")
    print(weather_data[['date', 'city', 'temp_mean_c', 'precip_mm', 'wind_max_ms']].head(10))
    
    # Prepare for integration (set index by date)
    weather_data = weather_data.set_index('date')
    
    print("Date range:", weather_data.index.min().strftime('%Y-%m-%d'), "to", weather_data.index.max().strftime('%Y-%m-%d'))
    print("Cities:", weather_data['city'].nunique())
else:
    print("No weather data available.")


Weather Data Sample:
        date      city  temp_mean_c  precip_mm  wind_max_ms
0 2007-07-30  New York         24.3        5.9          7.2
1 2007-07-31  New York         25.0        0.1         11.6
2 2007-08-01  New York         25.9        0.0          8.6
3 2007-08-02  New York         27.6        0.0         12.4
4 2007-08-03  New York         27.7        0.3         13.7
5 2007-08-04  New York         27.8        0.2         13.4
6 2007-08-05  New York         25.0        0.0         15.0
7 2007-08-06  New York         24.8        3.3         14.3
8 2007-08-07  New York         27.4        0.0         11.5
9 2007-08-08  New York         28.4        9.6         18.4
Date range: 2007-07-30 to 2025-09-13
Cities: 6


In [34]:
# # NOAA-compatible helper (overrides any previous definition)

# def get_weather_data_for_analysis(start_date, end_date, cities=None, usa_only=False):
#     """Fetch NOAA weather for supported cities over a date range."""
#     # Use only cities that have mapped stations
#     if usa_only or not cities:
#         target_cities = ['Houston', 'Dallas', 'Denver', 'New York', 'Los Angeles', 'Chicago']
#     else:
#         target_cities = [c for c in cities if c in weather_fetcher.weather_stations]
#         if not target_cities:
#             target_cities = ['Houston', 'Dallas', 'Denver', 'New York', 'Los Angeles', 'Chicago']
#     return weather_fetcher.get_multiple_stations_weather(target_cities, start_date, end_date)


In [35]:
# Integrate Open-Meteo weather with oil dataframe
def integrate_weather_with_oil_data(oil_data, weather_data):
    if weather_data.empty:
        print("No weather data to integrate")
        return oil_data
    if not isinstance(oil_data.index, pd.DatetimeIndex):
        oil_data.index = pd.to_datetime(oil_data.index)
    if not isinstance(weather_data.index, pd.DatetimeIndex):
        weather_data['date'] = pd.to_datetime(weather_data['date'])
        weather_data = weather_data.set_index('date')

    # Aggregate per day across cities
    weather_daily = weather_data.groupby(weather_data.index.date).agg({
        'temp_mean_c': ['mean', 'std', 'min', 'max'],
        'precip_mm': ['mean', 'sum', 'max'],
        'wind_max_ms': ['mean', 'max']
    }).round(3)
    
    weather_daily.columns = ['_'.join(col).strip() for col in weather_daily.columns]
    weather_daily.index = pd.to_datetime(weather_daily.index)
    combined = oil_data.merge(weather_daily, left_index=True, right_index=True, how='left')
    print("Integrated weather data with oil data")
    return combined

# Example integration with daily contiguous index from oil data
if 'weather_data' in locals() and not weather_data.empty and 'raw_data' in locals():
    combined_data = integrate_weather_with_oil_data(raw_data, weather_data)
    # Reindex to oil dates to ensure adjacency/contiguity
    combined_data = combined_data.reindex(raw_data.index)
    weather_cols = [c for c in combined_data.columns if any(x in c for x in ['temp_mean_c', 'precip_mm', 'wind_max_ms'])]
    print("Weather columns added:", len(weather_cols))
    print(combined_data[['wti_price', 'brent_price'] + weather_cols[:3]].head())
else:
    print("Weather data not available for integration")


Integrated weather data with oil data
Weather columns added: 9
            wti_price  brent_price  temp_mean_c_mean  temp_mean_c_std  \
Date                                                                    
2007-07-30  76.830002    75.739998            22.583            4.625   
2007-07-31  78.209999    77.050003            23.583            4.367   
2007-08-01  76.529999    75.349998            24.467            3.875   
2007-08-02  76.860001    75.760002            24.733            4.269   
2007-08-03  75.480003    74.750000            24.967            3.988   

            temp_mean_c_min  
Date                         
2007-07-30             14.1  
2007-07-31             15.3  
2007-08-01             17.0  
2007-08-02             16.3  
2007-08-03             17.0  


In [36]:
combined_data

,brent_price,wti_price,dxy,10y_yield,vix,eia_inventories,cpi,industrial_production,temp_mean_c_mean,temp_mean_c_std,temp_mean_c_min,temp_mean_c_max,precip_mm_mean,precip_mm_sum,precip_mm_max,wind_max_ms_mean,wind_max_ms_max
Date,,,,,,,,,,,,,,,,,
2007-07-30,75.739998,76.830002,80.849998,4.804,20.870001,NaN,NaN,NaN,22.583,4.625,14.1,27.6,5.300,31.8,25.8,12.517,20.4
2007-07-31,77.050003,78.209999,80.769997,4.771,23.520000,NaN,250.750811,100.132497,23.583,4.367,15.3,28.2,0.017,0.1,0.1,11.083,12.9
2007-08-01,75.349998,76.529999,80.870003,4.759,23.670000,502.074518,NaN,NaN,24.467,3.875,17.0,27.8,0.567,3.4,3.4,12.600,24.9
2007-08-02,75.760002,76.860001,80.709999,4.753,21.219999,NaN,NaN,NaN,24.733,4.269,16.3,27.6,3.250,19.5,19.1,14.550,26.4
2007-08-03,74.750000,75.480003,80.180000,4.700,25.160000,NaN,NaN,NaN,24.967,3.988,17.0,27.7,0.333,2.0,1.5,18.050,38.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-09-08,66.019997,62.259998,97.449997,4.046,15.110000,NaN,NaN,NaN,21.300,5.644,15.1,28.8,0.350,2.1,1.2,10.700,13.4
2025-09-09,66.389999,62.630001,97.790001,4.074,15.040000,NaN,NaN,NaN,20.883,5.017,15.7,28.6,0.133,0.8,0.5,10.683,12.8
2025-09-10,67.489998,63.669998,97.779999,4.032,15.350000,889.995752,NaN,NaN,21.533,4.855,15.6,28.2,2.800,16.8,7.7,12.317,19.4


In [37]:
death_file_path = 'ucdp-brd-conf-251-csv.zip'
#Load UCDP conflict-level data
df_deaths = pd.read_csv(death_file_path)

pd.set_option('display.max_columns', None)

df_deaths.columns


Index(['conflict_id', 'dyad_id', 'location_inc', 'side_a', 'side_a_id',
       'side_a_2nd', 'side_b', 'side_b_id', 'side_b_2nd', 'incompatibility',
       'territory_name', 'year', 'bd_best', 'bd_low', 'bd_high',
       'type_of_conflict', 'battle_location', 'gwno_a', 'gwno_a_2nd', 'gwno_b',
       'gwno_b_2nd', 'gwno_loc', 'gwno_battle', 'region', 'version'],
      dtype='object')

In [ ]:
import pandas as pd

def merge_daily_brd_features(combined_data, death_file_path, 
                             death_threshold=1000,  # yearly deaths threshold for major war
                             rolling_windows=[7,30,90],
                             conflict_types=None):
    """
    Merge daily war intensity and flags from UCDP Battle-Related Deaths into a daily oil price dataframe.
    
    Parameters:
    - combined_data: pd.DataFrame, must have DatetimeIndex
    - brd_filepath: str, path to BRD CSV
    - death_threshold: int, yearly deaths threshold to flag major war
    - rolling_windows: list of ints, window sizes for rolling averages
    - conflict_types: list of ints, optional, filter type_of_conflict
                      (1=state-based,2=non-state,3=one-sided)
    
    Returns:
    - combined_data merged with daily BRD features
    """
    
    # Load BRD dataset
    df_brd = pd.read_csv(brd_filepath)
    
    # Optional: filter by type_of_conflict
    if conflict_types is not None:
        df_brd = df_brd[df_brd['type_of_conflict'].isin(conflict_types)]
    
    # Aggregate total deaths per year across all dyads
    yearly_deaths = df_brd.groupby('year', as_index=False)['bd_best'].sum()
    
    # Create yearly binary war flag
    yearly_deaths['war'] = (yearly_deaths['bd_best'] >= death_threshold).astype(int)
    
    # Create daily index matching combined_data
    daily_index = pd.date_range(start=combined_data.index.min(), end=combined_data.index.max(), freq='D')
    daily_war = pd.DataFrame({'date': daily_index})
    daily_war['year'] = daily_war['date'].dt.year
    
    # Merge yearly deaths onto daily index
    daily_war = daily_war.merge(yearly_deaths[['year', 'bd_best', 'war']], on='year', how='left')
    
    # Fill missing years with 0
    daily_war['bd_best'] = daily_war['bd_best'].fillna(0).astype(int)
    daily_war['war'] = daily_war['war'].fillna(0).astype(int)
    daily_war = daily_war.drop(columns='year')
    
    # Compute rolling averages
    for window in rolling_windows:
        daily_war[f'deaths_roll_{window}d'] = daily_war['bd_best'].rolling(window=window, min_periods=1).mean()
    
    # Set index
    daily_war = daily_war.set_index('date')
    
    # Merge with combined_data
    merged = combined_data.join(daily_war, how='left')
    
    # Fill any remaining NaNs with 0
    merged[['bd_best','war'] + [f'deaths_roll_{w}d' for w in rolling_windows]] = \
        merged[['bd_best','war'] + [f'deaths_roll_{w}d' for w in rolling_windows]].fillna(0)
    
    return merged


In [41]:
combined_data = merge_daily_brd_features(combined_data)

TypeError: merge_daily_brd_features() missing 1 required positional argument: 'brd_filepath'